In [1]:
# for gpu only
!pip uninstall xgboost -y

Found existing installation: xgboost 3.1.0
Uninstalling xgboost-3.1.0:
  Successfully uninstalled xgboost-3.1.0


In [2]:
!pip install xgboost>=2.0.0

In [3]:
!pip uninstall lightgbm -y

Found existing installation: lightgbm 4.6.0
Uninstalling lightgbm-4.6.0:
  Successfully uninstalled lightgbm-4.6.0


In [4]:
!pip install lightgbm>=4.0.0

In [5]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.option_context('mode.use_inf_as_na', True)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import (accuracy_score, confusion_matrix,
                            classification_report, roc_curve,
                            roc_auc_score, precision_score,
                            recall_score, f1_score)
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline

import joblib
import optuna

import xgboost as xgb
import lightgbm as lgb
import catboost as cb  # Added CatBoost
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings("ignore")

rand_seed = 7

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [6]:
base_dir = '/kaggle/input/playground-series-s5e12'

df_train = pd.read_csv(os.path.join(base_dir, "train.csv"))
df_test = pd.read_csv(os.path.join(base_dir, "test.csv"))
df_submission = pd.read_csv(os.path.join(base_dir, "sample_submission.csv"))

In [7]:
print(f"Training data size: {df_train.shape}")
print(f"Testg data size: {df_test.shape}")

Training data size: (700000, 26)
Testg data size: (300000, 25)


In [8]:
df_train.head()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,70,62,199,58,114,102,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,77,71,199,50,121,124,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,89,73,188,59,114,108,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,69,74,182,54,85,123,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,60,85,206,49,131,124,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  700000 non-null  int64  
 1   age                                 700000 non-null  int64  
 2   alcohol_consumption_per_week        700000 non-null  int64  
 3   physical_activity_minutes_per_week  700000 non-null  int64  
 4   diet_score                          700000 non-null  float64
 5   sleep_hours_per_day                 700000 non-null  float64
 6   screen_time_hours_per_day           700000 non-null  float64
 7   bmi                                 700000 non-null  float64
 8   waist_to_hip_ratio                  700000 non-null  float64
 9   systolic_bp                         700000 non-null  int64  
 10  diastolic_bp                        700000 non-null  int64  
 11  heart_rate                

In [10]:
df_train.isna().sum()

id                                    0
age                                   0
alcohol_consumption_per_week          0
physical_activity_minutes_per_week    0
diet_score                            0
sleep_hours_per_day                   0
screen_time_hours_per_day             0
bmi                                   0
waist_to_hip_ratio                    0
systolic_bp                           0
diastolic_bp                          0
heart_rate                            0
cholesterol_total                     0
hdl_cholesterol                       0
ldl_cholesterol                       0
triglycerides                         0
gender                                0
ethnicity                             0
education_level                       0
income_level                          0
smoking_status                        0
employment_status                     0
family_history_diabetes               0
hypertension_history                  0
cardiovascular_history                0


In [11]:
df_test.isna().sum()

id                                    0
age                                   0
alcohol_consumption_per_week          0
physical_activity_minutes_per_week    0
diet_score                            0
sleep_hours_per_day                   0
screen_time_hours_per_day             0
bmi                                   0
waist_to_hip_ratio                    0
systolic_bp                           0
diastolic_bp                          0
heart_rate                            0
cholesterol_total                     0
hdl_cholesterol                       0
ldl_cholesterol                       0
triglycerides                         0
gender                                0
ethnicity                             0
education_level                       0
income_level                          0
smoking_status                        0
employment_status                     0
family_history_diabetes               0
hypertension_history                  0
cardiovascular_history                0


In [12]:
df_train.head()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,gender,ethnicity,education_level,income_level,smoking_status,employment_status,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,70,62,199,58,114,102,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,77,71,199,50,121,124,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,89,73,188,59,114,108,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,69,74,182,54,85,123,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,60,85,206,49,131,124,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0


In [13]:
df_train.describe()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
count,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000
mean,349999.500000,50.359734,2.072411,80.230803,5.963695,7.002200,6.012733,25.874684,0.858766,116.294193,75.440924,70.167749,186.818801,53.823214,102.905854,123.081850,0.149401,0.181990,0.030324,0.623296
std,202072.738554,11.655520,1.048189,51.195071,1.463336,0.901907,2.022707,2.860705,0.037980,11.010390,6.825775,6.938722,16.730832,8.266545,19.022416,24.739397,0.356484,0.385837,0.171478,0.484560
min,0.000000,19.000000,1.000000,1.000000,0.100000,3.100000,0.600000,15.100000,0.680000,91.000000,51.000000,42.000000,117.000000,21.000000,51.000000,31.000000,0.000000,0.000000,0.000000,0.000000
25%,174999.750000,42.000000,1.000000,49.000000,5.000000,6.400000,4.600000,23.900000,0.830000,108.000000,71.000000,65.000000,175.000000,48.000000,89.000000,106.000000,0.000000,0.000000,0.000000,0.000000
50%,349999.500000,50.000000,2.000000,71.000000,6.000000,7.000000,6.000000,25.900000,0.860000,116.000000,75.000000,70.000000,187.000000,54.000000,103.000000,123.000000,0.000000,0.000000,0.000000,1.000000
75%,524999.250000,58.000000,3.000000,96.000000,7.000000,7.600000,7.400000,27.800000,0.880000,124.000000,80.000000,75.000000,199.000000,59.000000,116.000000,139.000000,0.000000,0.000000,0.000000,1.000000
max,699999.000000,89.000000,9.000000,747.000000,9.900000,9.900000,16.500000,38.400000,1.050000,163.000000,104.000000,101.000000,289.000000,90.000000,205.000000,290.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
string_data = df_train.select_dtypes(include=['object'])
str_cols = string_data.columns.tolist()
num_cols = df_train.select_dtypes(include=['number']).columns.tolist()
print("Numerical Columns:", len(num_cols))
for i in num_cols:
    print(i, end=', ')
print("\nString Columns: ", len(str_cols))
for i in str_cols:
    print(i, end = ", ")

Numerical Columns: 20
id, age, alcohol_consumption_per_week, physical_activity_minutes_per_week, diet_score, sleep_hours_per_day, screen_time_hours_per_day, bmi, waist_to_hip_ratio, systolic_bp, diastolic_bp, heart_rate, cholesterol_total, hdl_cholesterol, ldl_cholesterol, triglycerides, family_history_diabetes, hypertension_history, cardiovascular_history, diagnosed_diabetes, 
String Columns:  6
gender, ethnicity, education_level, income_level, smoking_status, employment_status, 

In [15]:
df_train[str_cols].head()

,gender,ethnicity,education_level,income_level,smoking_status,employment_status
0,Female,Hispanic,Highschool,Lower-Middle,Current,Employed
1,Female,White,Highschool,Upper-Middle,Never,Employed
2,Male,Hispanic,Highschool,Lower-Middle,Never,Retired
3,Female,White,Highschool,Lower-Middle,Current,Employed
4,Male,White,Highschool,Upper-Middle,Never,Retired


Ordinal Features are: education_level, income_level

Nominal Features are: gender,	ethnicity,smoking_status,	employment_status

In [16]:
df_train[num_cols].head()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,70,62,199,58,114,102,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,77,71,199,50,121,124,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,89,73,188,59,114,108,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,69,74,182,54,85,123,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,60,85,206,49,131,124,0,1,0,1.0


In [17]:
class OneHotFeatureEncoder:
    def __init__(self):
        self.dir_path = None
        self.ohe = None
        self.columns = []
    
    def fit(self, data, columns):
        self.columns = columns
        self.ohe = OneHotEncoder(
            handle_unknown='ignore', # CRITICAL: Ignore categories not seen during fit
            sparse_output=False
        )
        self.ohe.fit(data[self.columns])

    def transform(self, data):
        res = self.ohe.transform(data[self.columns])
        return pd.DataFrame(
            data = res,
            columns = self.ohe.get_feature_names_out() 
        )

    def save(self, data_path):
        joblib.dump(self.columns, os.path.join(dir_path, 'ohe_columns.joblib'))
        joblib.dump(self.ohe, os.path.join(dir_path, 'ohe_encoder.joblib'))

    def load(self, data_path):
        self.ohe = joblib.load(os.path.join(dir_path, 'ohe_encoder.joblib'))
        self.columns = joblib.load(os.path.join(dir_path, 'ohe_columns.joblib'))




class OrdinalFeatureEncoder:
    def __init__(self):
        self.dir_path = None
        self.ode = None
        self.columns = []
    def fit(self, data, columns):
        self.columns = columns
        self.ode = OrdinalEncoder( )
        self.ode.fit(data[self.columns])

    def transform(self, data):
        res = self.ode.transform(data[self.columns])
        return pd.DataFrame(
            data = res,
            columns = self.ode.get_feature_names_out() 
        )

    def save(self, dir_path):
        joblib.dump(self.columns, os.path.join(dir_path, 'ode_columns.joblib'))
        joblib.dump(self.ode, os.path.join(dir_path, 'ode_encoder.joblib'))

    def load(self, dir_path):
        self.ode = joblib.load(os.path.join(dir_path, 'ode_encoder.joblib'))
        self.columns = joblib.load(os.path.join(dir_path, 'ode_columns.joblib'))


class CustomScaler:
    def __init__(self):
        self.scaler = None

    def fit(self, data):
        self.scaler = StandardScaler()
        self.scaler.fit(data)

    def transform(self, data):
        return self.scaler.transform(data)

    def save(self, dir_path):
        joblib.dump(self.scaler, os.path.join(dir_path, 'scaler.joblib'))

    def load(self, dir_path):
        self.scaler = joblib.load(os.path.join(dir_path, 'scaler.joblib'))

In [18]:
dir_path = '/kaggle/working'

In [19]:
def create_encoders(data, dir_path, ordinal_columns=[], onehot_columns=[]):
    eod = OrdinalFeatureEncoder()
    eod.fit(df_train, ordinal_columns)
    eod.save(dir_path)
    
    enc = OneHotFeatureEncoder()
    enc.fit(df_train, onehot_columns)
    enc.save(dir_path)

def create_features_scaler(data, dir_path):
    custom_scaler = CustomScaler()
    custom_scaler.fit(X)
    custom_scaler.save(dir_path)

def transform_features(data:pd.DataFrame, ordinal_encoder, onehot_encoder):
    res_ordinal_encoder = ordinal_encoder.transform(data)
    res_onehot_encoder = onehot_encoder.transform(data)

    all_columns = ordinal_encoder.columns + onehot_encoder.columns
    print(all_columns)
    
    return pd.concat([data, res_ordinal_encoder, res_onehot_encoder], axis=1).drop(columns=all_columns)


In [20]:
create_encoders(df_train, 
                dir_path, 
                ordinal_columns=['education_level','income_level'], 
                onehot_columns=['gender', 'ethnicity', 'smoking_status', 'employment_status'])

In [21]:
ordinal_encoder = OrdinalFeatureEncoder()
ordinal_encoder.load(dir_path)

onehot_encoder = OneHotFeatureEncoder()
onehot_encoder.load(dir_path)

In [22]:
data = transform_features(df_train, ordinal_encoder, onehot_encoder)

['education_level', 'income_level', 'gender', 'ethnicity', 'smoking_status', 'employment_status']


In [23]:
data.head()

,id,age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,family_history_diabetes,hypertension_history,cardiovascular_history,diagnosed_diabetes,gender_Female,gender_Male,gender_Other,ethnicity_Asian,ethnicity_Black,ethnicity_Hispanic,ethnicity_Other,ethnicity_White,smoking_status_Current,smoking_status_Former,smoking_status_Never,employment_status_Employed,employment_status_Retired,employment_status_Student,employment_status_Unemployed
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,70,62,199,58,114,102,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,77,71,199,50,121,124,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,89,73,188,59,114,108,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,69,74,182,54,85,123,0,1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,60,85,206,49,131,124,0,1,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [24]:
X = data.drop(['id','diagnosed_diabetes'],axis = 1) 
y = data['diagnosed_diabetes']

In [25]:
create_features_scaler(X, dir_path)

In [26]:
custom_scaler = CustomScaler()
custom_scaler.load(dir_path)

X_scaled = custom_scaler.transform(X)

In [27]:
#1/0

In [28]:
# X = np.random.rand(10000, 50)
# y = np.random.randint(0, 2, 10000)

# model = xgb.XGBClassifier(
#     tree_method="hist",
#     device="cuda",
#     n_estimators=100
# )

# model.fit(X, y)
# print("✅ GPU training successful")

In [29]:
print(xgb.__version__)
print(xgb.get_config())

3.1.2
{'nthread': 0, 'use_rmm': False, 'verbosity': 1}


#### Hyperparameter Tuning (Optuna) of XGBoost

In [30]:
# -------------------------------------------------
# Unified objective function
# -------------------------------------------------
def objective(trial, X, y, model_type="xgb", use_gpu=False):

    scale_pos_weight = (y == 0).sum() / (y == 1).sum()
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # =======================
        # XGBOOST
        # =======================
        if model_type == "xgb":
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 300, 2000),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                "gamma": trial.suggest_float("gamma", 1e-8, 10.0, log=True),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
                "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
                "objective": "binary:logistic",
                "eval_metric": "auc",
                "scale_pos_weight": scale_pos_weight,
                "tree_method": "hist",
                "device": "cuda" if use_gpu else "cpu",
                "random_state": 42,
                "verbosity": 0
            }

            model = XGBClassifier(**params)

            model.fit(
                X_train,
                y_train,
                eval_set=[(X_valid, y_valid)],
                verbose=False
            )

        # =======================
        # LIGHTGBM
        # =======================
        elif model_type == "lgbm":
            params = {
                "n_estimators": trial.suggest_int("n_estimators", 300, 2000),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "num_leaves": trial.suggest_int("num_leaves", 31, 255),
                "max_depth": trial.suggest_int("max_depth", 3, 12),
                "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
                "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
                "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
                "objective": "binary",
                "metric": "auc",
                "scale_pos_weight": scale_pos_weight,
                "device": "gpu" if use_gpu else "cpu",
                "verbosity": -1,
                "random_state": 42
            }

            model = LGBMClassifier(**params)

            model.fit(
                X_train,
                y_train,
                eval_set=[(X_valid, y_valid)],
                eval_metric="auc"
            )

        # =======================
        # CATBOOST
        # =======================
        elif model_type == "cat":
            params = {
                "iterations": trial.suggest_int("iterations", 300, 2000),
                "depth": trial.suggest_int("depth", 4, 10),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
                "random_strength": trial.suggest_float("random_strength", 1e-3, 10.0, log=True),
                "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
                "loss_function": "Logloss",
                "eval_metric": "AUC",
                "scale_pos_weight": scale_pos_weight,
                "task_type": "GPU" if use_gpu else "CPU",
                "devices": "0",
                "bootstrap_type": "Bayesian",
                "random_seed": 42,
                "verbose": False
            }

            model = CatBoostClassifier(**params)

            model.fit(
                X_train,
                y_train,
                eval_set=(X_valid, y_valid),
                early_stopping_rounds=50,
                use_best_model=True
            )

        else:
            raise ValueError("Invalid model_type")

        preds = model.predict_proba(X_valid)[:, 1]
        scores.append(roc_auc_score(y_valid, preds))

    return np.mean(scores)


# -------------------------------------------------
# Tuner API
# -------------------------------------------------
def tune_model(X, y, model_type="xgb", use_gpu=False, n_trials=50):

    study = optuna.create_study(
        direction="maximize",
        study_name=f"{model_type}_optuna",
        pruner=optuna.pruners.MedianPruner()
    )

    study.optimize(
        lambda trial: objective(trial, X, y, model_type, use_gpu),
        n_trials=n_trials,
        show_progress_bar=True
    )

    print("\n🔥 Best ROC-AUC:", study.best_value)
    print("🔥 Best Params:")
    for k, v in study.best_params.items():
        print(f"  {k}: {v}")

    return study


In [31]:
# XGBoost GPU
study_xgb = tune_model(X_scaled, y, model_type="xgb", use_gpu=True, n_trials=1)

[I 2025-12-18 14:44:56,961] A new study created in memory with name: xgb_optuna


  0%|          | 0/1 [00:00<?, ?it/s]

[I 2025-12-18 14:45:44,917] Trial 0 finished with value: 0.7250183724190443 and parameters: {'n_estimators': 1130, 'max_depth': 8, 'learning_rate': 0.03649204909912953, 'subsample': 0.8258951425605852, 'colsample_bytree': 0.8726313583364063, 'gamma': 0.11337760553169023, 'min_child_weight': 2, 'reg_alpha': 0.8048969228990768, 'reg_lambda': 0.0008361851919314513}. Best is trial 0 with value: 0.7250183724190443.

🔥 Best ROC-AUC: 0.7250183724190443
🔥 Best Params:
  n_estimators: 1130
  max_depth: 8
  learning_rate: 0.03649204909912953
  subsample: 0.8258951425605852
  colsample_bytree: 0.8726313583364063
  gamma: 0.11337760553169023
  min_child_weight: 2
  reg_alpha: 0.8048969228990768
  reg_lambda: 0.0008361851919314513


In [32]:
# LightGBM CPU
study_lgbm = tune_model(X_scaled, y, model_type="lgbm", use_gpu=False, n_trials=1)

[I 2025-12-18 14:45:44,948] A new study created in memory with name: lgbm_optuna


  0%|          | 0/1 [00:00<?, ?it/s]

[I 2025-12-18 14:48:45,583] Trial 0 finished with value: 0.7264181621663358 and parameters: {'n_estimators': 660, 'learning_rate': 0.05336574494163678, 'num_leaves': 71, 'max_depth': 9, 'min_child_samples': 42, 'subsample': 0.7096210156787649, 'colsample_bytree': 0.6358750320167348, 'reg_alpha': 2.513602313364552e-05, 'reg_lambda': 0.0006408084980160521}. Best is trial 0 with value: 0.7264181621663358.

🔥 Best ROC-AUC: 0.7264181621663358
🔥 Best Params:
  n_estimators: 660
  learning_rate: 0.05336574494163678
  num_leaves: 71
  max_depth: 9
  min_child_samples: 42
  subsample: 0.7096210156787649
  colsample_bytree: 0.6358750320167348
  reg_alpha: 2.513602313364552e-05
  reg_lambda: 0.0006408084980160521


In [33]:
# CatBoost GPU
study_cat = tune_model(X_scaled, y, model_type="cat", use_gpu=True, n_trials=1)

[I 2025-12-18 14:48:45,615] A new study created in memory with name: cat_optuna


  0%|          | 0/1 [00:00<?, ?it/s]

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


[I 2025-12-18 14:50:12,791] Trial 0 finished with value: 0.7117220032889209 and parameters: {'iterations': 445, 'depth': 6, 'learning_rate': 0.02034897778493428, 'l2_leaf_reg': 0.013332731004810132, 'random_strength': 0.05093860902687294, 'bagging_temperature': 0.6651793408329196}. Best is trial 0 with value: 0.7117220032889209.

🔥 Best ROC-AUC: 0.7117220032889209
🔥 Best Params:
  iterations: 445
  depth: 6
  learning_rate: 0.02034897778493428
  l2_leaf_reg: 0.013332731004810132
  random_strength: 0.05093860902687294
  bagging_temperature: 0.6651793408329196


In [34]:
final_xgb_model = XGBClassifier(
    **study_xgb.best_params,
    tree_method="hist",
    device="cuda",
    eval_metric="auc"
)

final_xgb_model.fit(X_scaled, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8726313583364063, device='cuda',
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, feature_weights=None,
              gamma=0.11337760553169023, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03649204909912953,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1130, n_jobs=None,
              num_parallel_tree=None, ...)

In [35]:
final_catboost_model = CatBoostClassifier(
    **study_cat.best_params, 
    verbose=False
)
final_catboost_model.fit(X_scaled, y)

In [36]:
final_lgbm_model = LGBMClassifier(
    **study_lgbm.best_params
)

final_lgbm_model.fit(X_scaled,y)

LGBMClassifier(colsample_bytree=0.6358750320167348,
               learning_rate=0.05336574494163678, max_depth=9,
               min_child_samples=42, n_estimators=660, num_leaves=71,
               reg_alpha=2.513602313364552e-05,
               reg_lambda=0.0006408084980160521, subsample=0.7096210156787649)

In [37]:
def predict(model, data):
    ordinal_encoder = OrdinalFeatureEncoder()
    ordinal_encoder.load(dir_path)
    
    onehot_encoder = OneHotFeatureEncoder()
    onehot_encoder.load(dir_path)

    data = transform_features(data, ordinal_encoder, onehot_encoder)

    custom_scaler = CustomScaler()
    custom_scaler.load(dir_path)
    
    X_scaled = custom_scaler.transform(data)

    #res = model.predict(X_scaled)
    res = model.predict_proba(X_scaled)[:,1]

    return res

In [38]:
res_xgb = predict(final_xgb_model, df_train.drop(columns=['id', 'diagnosed_diabetes']))
res_catboost = predict(final_catboost_model, df_train.drop(columns=['id', 'diagnosed_diabetes']))
res_lgbm = predict(final_lgbm_model, df_train.drop(columns=['id', 'diagnosed_diabetes']))

w_lgb = 0.30
w_xgb = 0.30
w_cat = 0.40

final_res = (w_lgb * res_lgbm) + (w_xgb * res_xgb) + (w_cat * res_catboost)

ensemble_score = roc_auc_score(y, final_res)
print(f"Weighted Ensemble AUC: {ensemble_score:.5f}")


['education_level', 'income_level', 'gender', 'ethnicity', 'smoking_status', 'employment_status']
['education_level', 'income_level', 'gender', 'ethnicity', 'smoking_status', 'employment_status']
['education_level', 'income_level', 'gender', 'ethnicity', 'smoking_status', 'employment_status']
Weighted Ensemble AUC: 0.75905


In [39]:
res_xgb = predict(final_xgb_model, df_test.drop(columns=['id']))
res_catboost = predict(final_catboost_model, df_test.drop(columns=['id']))
res_lgbm = predict(final_lgbm_model, df_test.drop(columns=['id']))

final_res = (w_lgb * res_lgbm) + (w_xgb * res_xgb) + (w_cat * res_catboost)

df_sub = pd.DataFrame(data={
    'id': df_test['id'],
    'diagnosed_diabetes': final_res
})
df_sub.head()
df_sub.to_csv(os.path.join("submission_ens.csv"), index=False)

['education_level', 'income_level', 'gender', 'ethnicity', 'smoking_status', 'employment_status']
['education_level', 'income_level', 'gender', 'ethnicity', 'smoking_status', 'employment_status']
['education_level', 'income_level', 'gender', 'ethnicity', 'smoking_status', 'employment_status']
